In [1]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.Collecting gradio
  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/27/0f/9ebe13938b1357123cffa630468d390d8fbbf344bba27441b97fd491555e/gradio-4.22.0-py3-none-any.whl.metadata
  Using cached gradio-4.22.0-py3-none-any.whl.metadata (15 kB)
  Obtaining dependency information for altair<6.0,>=4.2.0 from https://files.pythonhosted.org/packages/c5/e4/7fcceef127badbb0d644d730d992410e4f3799b295c9964a172f92a469c7/altair-5.2.0-py3-none-any.whl.metadata
  Using cached altair-5.2.0-py3-none-any.whl.metadata (8.7 kB)
  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/f0/f7/ea860cb8aa18e326f411e32ab537424690a53db20de6bad73d70da611fae/fastapi-0.110.0-py3-none-any.whl.metadata
  Using cached fastapi-0.110.0-py3-none-any.whl.metadata (25 kB)
  Obtaining dependency information for gradio-client==0.13.0 from https://files.pythonhosted.org/packages/93/4e/a9b825


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import gradio as gr

d:\Capstone\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def prompt_paint(input_image, source_prompt, result_prompt):
  
  #input_image = Image.open(requests.get(image_url, stream=True).raw)

  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      transforms.Resize((512, 512)),
  ])
  
  img = transform(input_image).unsqueeze(0)

  prompts = [source_prompt]

  # predict
  with torch.no_grad():
      preds = model(img.repeat(len(prompts),1,1,1), prompts)[0]


  input_image.convert('RGB').resize((512, 512)).save("init_image.png", "PNG")
  
  
  filename = f"mask.png"
  
  plt.imsave(filename,torch.sigmoid(preds[0][0]))

  img2 = cv2.imread(filename)

  gray_image = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  (thresh, bw_image) = cv2.threshold(gray_image, 100, 255, cv2.THRESH_BINARY)

  # For debugging only:
  cv2.imwrite(filename,bw_image)

  # fix color format
  cv2.cvtColor(bw_image, cv2.COLOR_BGR2RGB)

  Image.fromarray(bw_image)

  init_image = Image.open('init_image.png')
  mask = Image.open('mask.png')

  with autocast("cuda"):
    images = pipe(prompt = result_prompt, init_image=init_image, mask_image=mask, strength=0.8)["sample"]

  return(images[0])


In [4]:
def prompt_paint_advanced(input_image, source_prompt, result_prompt, mask_precision, sd_strength):
  
  #input_image = Image.open(requests.get(image_url, stream=True).raw)

  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      transforms.Resize((512, 512)),
  ])
  
  img = transform(input_image).unsqueeze(0)

  prompts = [source_prompt]

  # predict
  with torch.no_grad():
      preds = model(img.repeat(len(prompts),1,1,1), prompts)[0]


  input_image.convert('RGB').resize((512, 512)).save("init_image.png", "PNG")
  
  
  filename = f"mask.png"
  
  plt.imsave(filename,torch.sigmoid(preds[0][0]))

  img2 = cv2.imread(filename)

  gray_image = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  (thresh, bw_image) = cv2.threshold(gray_image, mask_precision, 255, cv2.THRESH_BINARY)

  # For debugging only:
  cv2.imwrite(filename,bw_image)

  # fix color format
  cv2.cvtColor(bw_image, cv2.COLOR_BGR2RGB)

  Image.fromarray(bw_image)

  init_image = Image.open('init_image.png')
  mask = Image.open('mask.png')

  with autocast("cuda"):
    images = pipe(prompt = result_prompt, init_image=init_image, mask_image=mask, strength=sd_strength)["sample"]

  return(images[0])

In [5]:
with gr.Blocks() as demo:
    gr.Markdown("# Prompt Paint - Stable Diffusion")
    with gr.Tab("Prompt Paint - Basic"):
        with gr.Row():
            

            input_image = gr.Image(label = 'Upload your input image', type = 'pil')

            source_prompt = gr.Textbox(label="What is in the input image you want to change?")
            result_prompt = gr.Textbox(label="Replace it with?")

            image_output = gr.Image()
        
        image_button = gr.Button("Generate")

        image_button.click(prompt_paint, inputs=[input_image, source_prompt, result_prompt ], outputs=image_output)

    with gr.Tab("Prompt Paint - Advanced"):
        with gr.Row():
            

            input_image = gr.Image(label = 'Upload your input image', type = 'pil')

            source_prompt = gr.Textbox(label="What is in the input image you want to change?")
            result_prompt = gr.Textbox(label="Replace it with?")

                        
            mask_precision = gr.Slider(minimum = 50, maximum = 255, value = 70, label = 'Mask Precision')

            sd_strength = gr.Slider(minimum = 0, maximum = 1, value = 0.8, step = 0.1, label = 'Stable Diffusion Strength')
            

            image_output = gr.Image()
        
        image_button_advanced = gr.Button("Generate")

        image_button_advanced.click(prompt_paint_advanced, inputs=[input_image, source_prompt, result_prompt, mask_precision, sd_strength ], outputs=image_output)

demo.launch(debug = True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
